In [0]:
##Please note that I am working out of the files instead of the tables. this is because I believe that this is the expected pattern. The tables are purely for publication purposes.
from pyspark.sql import functions as F, Window, types as T

# Parameters with default values. 
dbutils.widgets.text('catalog', 'rearcquest', 'Catalog')
catalog = dbutils.widgets.get('catalog') 

dbutils.widgets.text('sourceSchema', 'raw', 'SourceSchema')
sourceSchema = dbutils.widgets.get('sourceSchema')

dbutils.widgets.text('targetSchema', 'reports', 'SourceSchema')
targetSchema = dbutils.widgets.get('targetSchema')


# Volume paths
BlsVolume = f'/Volumes/{catalog}/{sourceSchema}/bls_data/'
PopulationVolume = f'/Volumes/{catalog}/{sourceSchema}/population/'

# File paths
BlsFile = f'{BlsVolume}pr.data.0.Current'
PopulationFile = f'{PopulationVolume}population_data.json'


In [0]:
%run ./Utils

In [0]:
ensure_path(catalog=catalog,schema=targetSchema,type='schema')

In [0]:
# Read BLS data from Unity Catalog Volume
blsDf = spark.read.option("header", True) \
    .option("sep", "\t") \
    .option("inferSchema", True) \
    .csv(BlsFile)

blsDf = blsDf.toDF(*[c.strip() for c in blsDf.columns])

# Trim string columns
stringColumns = [field.name for field in blsDf.schema.fields if isinstance(field.dataType, T.StringType)]

for colName in stringColumns:
    blsDf = blsDf.withColumn(colName, F.trim(F.col(colName)))

#blsDf.printSchema()
#blsDf.display()

In [0]:
# Aggregate BLS data - find best year for each series
sumPerYear = blsDf.groupBy("series_id", "year").agg(F.sum("value").alias("yearly_sum"))

# Get the year with maximum sum for each series
bestYear = sumPerYear.groupBy("series_id").agg(
    F.max_by("year", "yearly_sum").alias("year"),
    F.max("yearly_sum").alias("yearly_sum")
)

#bestYear.display()

# Write to Delta table in Unity Catalog
bestYear.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{targetSchema}.best_year")

print(f"Saved to table: {catalog}.{targetSchema}.best_year")

In [0]:
# Read population data from Unity Catalog Volume
populationDf = spark.read.option("multiline", True).json(PopulationFile)

populationDf = populationDf.select(F.explode("data").alias("data"))
populationDf = populationDf.select("data.*")

#populationDf.printSchema()
#populationDf.show()

In [0]:
# Filter population data and aggregate
populationDf = populationDf.filter((F.col("Year") >= 2013) & (F.col("Year") <= 2018))

populationStats = populationDf.agg(
    F.mean("Population").alias("mean_population"),
    F.stddev("Population").alias("std_population")
)

#populationStats.display()

# Write to Delta table in Unity Catalog
populationStats.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{targetSchema}.population_stats")

print(f"Saved to table: {catalog}.{targetSchema}.population_stats")

In [0]:
# Join the datasets and filter
targetSeries = "PRS30006032"
targetPeriod = "Q01"

report = blsDf.filter((F.col("series_id") == targetSeries) & (F.col("period") == targetPeriod)) \
              .join(populationDf, blsDf.year == populationDf.Year, how="left") \
              .select(blsDf["*"], "Population")

#report.display()

# Write to Delta table in Unity Catalog
report.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{targetSchema}.bls_with_population")

print(f"Saved to table: {catalog}.{targetSchema}.bls_with_population")